In [1]:
%pip install reportlab

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 📦 Imports
import pandas as pd
from tqdm import tqdm
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet

# ✅ Step 1: Load Phase 4A Output
df = pd.read_csv("phase4a_rewrites.csv")  # Output of Phase 4A
df = df[df['stereotype_type'] != 'none'].copy()

# ✅ Step 2: Add severity score
severity_map = {
    "appearance_focus": 2,
    "relationship_only": 2,
    "agency_gap": 3,
    "occupation_gap": 3,
}
df["severity_score"] = df["stereotype_type"].map(severity_map).fillna(1)

# ✅ Step 3: Use existing rewrites
df["rewrite"] = df["rewritten_line"]

# ✅ Step 4: Rank by severity
df["rank"] = df["severity_score"].rank(method="dense", ascending=False).astype(int)

# ✅ Step 5: Save intermediate report CSV
df.to_csv("feedback_report.csv", index=False)
print("✅ CSV report saved as feedback_report.csv")

# ✅ Step 6: Generate PDF Report
# PDF Setup
doc = SimpleDocTemplate("feedback_report.pdf", pagesize=A4)
styles = getSampleStyleSheet()
elements = []

# Title
elements.append(Paragraph("🎬 Bollywood Bias Buster – Script Feedback Report", styles["Title"]))
elements.append(Spacer(1, 12))

# Table Header
data = [["#", "Original Line", "Stereotype", "Severity", "Rewritten Line"]]

# Build table rows
for idx, row in df.sort_values("rank").iterrows():
    data.append([
        row["rank"],
        row["line"],
        row["stereotype_type"],
        int(row["severity_score"]),
        row["rewrite"]
    ])

# Format Table
table = Table(data, colWidths=[30, 160, 85, 60, 180])
table.setStyle(TableStyle([
    ("BACKGROUND", (0, 0), (-1, 0), colors.grey),
    ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
    ("ALIGN", (0, 0), (-1, -1), "LEFT"),
    ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
    ("FONTSIZE", (0, 0), (-1, -1), 7),
    ("BOTTOMPADDING", (0, 0), (-1, 0), 6),
    ("GRID", (0, 0), (-1, -1), 0.25, colors.black),
]))

# Highlight severe lines
for i in range(1, len(data)):
    if int(data[i][3]) >= 3:
        table.setStyle([("BACKGROUND", (0, i), (-1, i), colors.lightpink)])

# Add table to document
elements.append(table)
doc.build(elements)

print("✅ PDF report saved as feedback_report.pdf")


✅ CSV report saved as feedback_report.csv
✅ PDF report saved as feedback_report.pdf
